In [ ]:
def erro_absoluto(referencia, estimativa):
    return abs(referencia - estimativa)

def erro_relativo(referencia, estimativa):
    return abs((referencia, estimativa) / estimativa)


In [ ]:
def limite_iteracao(limite):
    return lambda estatistica: estatistica.iteracao == limite

In [ ]:
class EstatisticaBissecao:

    def __init__(self, iteracao, a, b, est_x, est_y, est_erro) -> None:
        self.iteracao = iteracao
        self.a = a
        self.b = b
        self.est_x = est_x
        self.est_y = est_y
        self.est_erro = est_erro
    
    def __str__(self) -> str:
        return f"iteracao: {self.iteracao}, a: {self.a:0.06}, b: {self.b}, est_x: {self.est_x}, est_y: {self.est_y}, est_erro: {self.est_erro}"


def bissecao(a, b, f, condicao_parada):
    if a > b:
        a, b = b, a
    i = 1
    est_x = (a+b) / 2
    est_y = f(est_x)
    est_erro = (b-a) / 2
    estatistica = EstatisticaBissecao(i, a, b, est_x, est_y, est_erro)
    print(estatistica)
    while not condicao_parada(estatistica):
        i += 1
        if f(a) * est_y < 0:
            b = est_x
        else:
            a = est_x
        est_x = (a+b) / 2
        est_y = f(est_x)
        est_erro = (b-a) / 2
        estatistica = EstatisticaBissecao(i, a, b, est_x, est_y, est_erro)
        print(estatistica)
    return estatistica


In [ ]:
def f(x):
    return 5 * x + 2


In [ ]:
resultado = bissecao(-10., 10., f, limite_iteracao(100))